## Phase 6 - BI Feed for Dashboard

### Import Statements

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")

In [2]:
PROC = Path("../data/processed")
FORE = Path("../data/forecasting")
OUT  = FORE

FORWARD_FILL_CONTEXT = True   
MERGE_MODE = "by_quarter"   

In [3]:
prophet = pd.read_csv(FORE/"forecast_results.csv")
hist    = pd.read_csv(PROC/"yearly_spend.csv")   

In [4]:
def canonize_dept(s: str) -> str:
    s = re.sub(r"\s+", " ", str(s))
    return s.strip()

prophet["department"] = prophet["department"].map(canonize_dept)
hist["department"]    = hist["department"].map(canonize_dept)

In [5]:
prophet["ds"]   = pd.to_datetime(prophet["ds"])
hist["year"]    = hist["year"].astype(int)
prophet["year"] = prophet["ds"].dt.year.astype(int)

if MERGE_MODE == "by_year":
    bi = prophet.merge(
        hist[["department","year","avg_total_comp","total_comp_sum","employee_count"]],
        on=["department","year"], how="left"
    ).drop(columns="year").sort_values(["department","ds"]).reset_index(drop=True)
elif MERGE_MODE == "by_quarter":
    # expand yearly history to quarter ends so ds aligns 1:1
    qends = ["-03-31","-06-30","-09-30","-12-31"]
    hist_q = pd.DataFrame(
        [{
            "department": r.department,
            "ds": pd.to_datetime(f"{int(r.year)}{q}"),
            "avg_total_comp": r.avg_total_comp,
            "total_comp_sum": r.total_comp_sum,
            "employee_count": r.employee_count
        } for r in hist.itertuples(index=False) for q in qends]
    )
    hist_q["department"] = hist_q["department"].map(canonize_dept)
    bi = prophet.merge(hist_q, on=["department","ds"], how="left") \
                .sort_values(["department","ds"]).reset_index(drop=True)
else:
    raise ValueError('MERGE_MODE must be "by_year" or "by_quarter"')

In [6]:
max_hist_year = hist.groupby("department")["year"].max()
bi["is_future"] = bi["ds"].dt.year.gt(bi["department"].map(max_hist_year))

In [7]:
if FORWARD_FILL_CONTEXT:
    cols = ["avg_total_comp","total_comp_sum","employee_count"]
    bi[cols] = (bi.sort_values(["department","ds"])
                  .groupby("department", group_keys=False)[cols]
                  .ffill())

In [8]:
assert not bi["department"].isna().any(), "Unexpected blank department labels after canonicalization."
if not FORWARD_FILL_CONTEXT:
    hist_mask = ~bi["is_future"]
    if bi.loc[hist_mask, ["avg_total_comp","employee_count"]].isna().any().any():
        print("Note: some historical rows have missing context. Check source coverage or use MERGE_MODE='by_quarter'.")

In [9]:
out_path = OUT/"bi_feed_compensation_forecasts.csv"
print("Saved the data to ../data/forecasting/bi_feed_compensation_forecasts.csv")
bi.to_csv(out_path, index=False)
print(f"Saved → {out_path}")
display(bi.head(20))
display(bi.isna().mean().round(3))

Saved the data to ../data/forecasting/bi_feed_compensation_forecasts.csv
Saved → ../data/forecasting/bi_feed_compensation_forecasts.csv


,ds,yhat,yhat_lower,yhat_upper,department,year,avg_total_comp,total_comp_sum,employee_count,is_future
0,2013-03-31,87489.708977,87489.708971,87489.708982,Admin Offices,2013,87487.075,174974.15,2.0,False
1,2013-06-30,87483.021118,87483.021112,87483.021124,Admin Offices,2013,87487.075,174974.15,2.0,False
2,2013-09-30,87485.980789,87485.980783,87485.980795,Admin Offices,2013,87487.075,174974.15,2.0,False
3,2013-12-31,87489.005469,87489.005464,87489.005475,Admin Offices,2013,87487.075,174974.15,2.0,False
4,2014-03-31,81167.168170,81167.168164,81167.168175,Admin Offices,2014,62197.000,186591.00,3.0,False
5,2014-06-30,74837.964173,74837.964168,74837.964179,Admin Offices,2014,62197.000,186591.00,3.0,False
6,2014-09-30,68518.318094,68518.318088,68518.318100,Admin Offices,2014,62197.000,186591.00,3.0,False
7,2014-12-31,62198.901191,62198.901185,62198.901197,Admin Offices,2014,62197.000,186591.00,3.0,False
8,2015-03-31,73856.372566,73856.372559,73856.372571,Admin Offices,2015,108824.094,544120.47,5.0,False
9,2015-06-30,85506.472371,85506.472365,85506.472376,Admin Offices,2015,108824.094,544120.47,5.0,False


ds                0.0
yhat              0.0
yhat_lower        0.0
yhat_upper        0.0
department        0.0
year              0.0
avg_total_comp    0.1
total_comp_sum    0.1
employee_count    0.1
is_future         0.0
dtype: float64